## Imports

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

import pickle
from multiprocessing import Pool

import modeling
import helpers
import scienceplots
plt.style.use(['science','no-latex'])
# plt.style.use(['science','ieee','no-latex'])

## Load data

In [ ]:
with open('data.pkl', 'rb') as file:
    data_dict = pickle.load(file)
    all_data = data_dict['data']
    meta_data = data_dict['meta_data']

## Fit models

In [ ]:
this_model = 'option_model'
structure = 'top_down'
num_subject = this_data['tr'].shape[0]

alpha_bounds = [1e-6, 1]
beta_bounds = [1e-6, 10]
concentration_bounds = [1e-6, 50]
epsilon_bounds = [1e-6, 0.998]
bounds = [alpha_bounds, beta_bounds, concentration_bounds, epsilon_bounds, epsilon_bounds] 
param_names = ["alpha_2", "beta_2", 'concentration_2', 'epsilon', 'prior']

best_llh_all = np.zeros(num_subject)
best_params_all = np.zeros((num_subject, len(param_names)))

D = helpers.get_model_fit_data(all_data, num_block=12)

if __name__ == "__main__":
    inputs = []
       
    for i in range(num_subject):
        inputs.append((this_model, D[D[:,0]==i,:], structure, i, bounds))

    with Pool() as p:
        results = p.map(modeling.parallel_worker, inputs)

    for i, best_params, best_llh in results:
        best_params_all[i, :], best_llh_all[i] = best_params, best_llh

np.save('fitting_results/'+structure+'_params.npy', best_params_all)
np.save('fitting_results/'+structure+'_llh.npy', best_llh_all)


In [ ]:
best_params_all = np.load('fitting_results/'+structure+'_params.npy')
best_llh_all = np.load('fitting_results/'+structure+'_llh.npy')

In [ ]:
niters_sim = 5
if __name__ == "__main__":
    inputs = []
    for i in range(best_params_all.shape[0]):
        alpha_2, beta_2, concentration_2, epsilon, prior = best_params_all[i,:]
        condition = meta_data['Condition'][i]
        inputs.append((this_model, i, niters_sim, 1, alpha_2, 10, beta_2, 10, concentration_2, epsilon, prior, condition))
        
    with Pool() as p:
        results = p.map(modeling.parallel_simulator, inputs)

    all_data_sim = {}
    for _, this_data in results:
        all_data_sim = helpers.concatenate_data(this_data, all_data_sim)

In [ ]:
trials_to_probe = 10

stage2_info_ca1 = helpers.extract_stage2_info(this_data,experiment=condition)

mean_population_counter1_ca1, mean_population_counter2_ca1 = helpers.calc_mean(this_data, start_trial=0, trials_to_probe=trials_to_probe)

mean_counter1_ca1 = np.mean(mean_population_counter1_ca1,axis=0)
mean_counter2_ca1 = np.nanmean(mean_population_counter2_ca1,axis=0)

se_counter1_ca1 = stats.sem(mean_population_counter1_ca1,axis=0,nan_policy='omit')
se_counter2_ca1 = stats.sem(mean_population_counter2_ca1,axis=0,nan_policy='omit')

stage2_info_ca1_sim = helpers.extract_stage2_info(all_data_sim,experiment=condition)

mean_population_counter1_ca1_sim, mean_population_counter2_ca1_sim = helpers.calc_mean(all_data_sim, start_trial=0, trials_to_probe=trials_to_probe)

mean_counter1_ca1_sim = np.mean(mean_population_counter1_ca1_sim,axis=0)
mean_counter2_ca1_sim = np.mean(mean_population_counter2_ca1_sim,axis=0)

se_counter1_ca1_sim = stats.sem(mean_population_counter1_ca1_sim,axis=0,nan_policy='omit')
se_counter2_ca1_sim = stats.sem(mean_population_counter2_ca1_sim,axis=0,nan_policy='omit')

blocks = range(1,13)
plt.fill_between([6.5,6.5,7.5],[1,3,3],color='lightcoral',alpha=0.15)
plt.fill_between([10.5,10.5,11.5],[1,3,3],color='lightcoral',alpha=0.15)
plt.plot(blocks,np.ones(12)*2.5,'--',color='gray',linewidth=2,alpha=0.6,label='Chance')
plt.errorbar(blocks,mean_counter2_ca1,se_counter2_ca1,fmt='-',capsize=4,color='green',alpha=0.75,label='Gen')
plt.plot(blocks,mean_counter2_ca1_sim,'--',color='green',alpha=0.75,label='Rec')
plt.ylim([1,2.7])
plt.ylabel('Number of key presses')
plt.xlabel('Block')
plt.title(condition+', trials 1-10 (n=' + str(num_subject) + ')')
plt.legend()
plt.show()